In [1]:
import os

In [2]:
%pwd

'd:\\Metro\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Metro'

In [12]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir : Path
    data_path: Path

In [13]:
from Trafficvol.constants import *
from Trafficvol.utils.common import read_yaml,create_directories

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [15]:
import os
from Trafficvol import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [16]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.x = None
        self.y = None


    
    def data_preprocessing(self):
        
        data = pd.read_csv(self.config.data_path)
        
        
        data.drop(columns = ["holiday",'weather_description'],axis = 1, inplace = True) 
        
        data = data[data["weather_main"].isin(["Clouds", "Clear", "Mist", "Rain", "Snow", "Drizzle", "Haze", "Thunderstorm", "Fog"])]
        
        
        data['date_time'] = pd.to_datetime(data['date_time'], format='%Y-%m-%d %H:%M:%S')

        # Extract Date Features
        data['year'] = data['date_time'].dt.year
        data['month'] = data['date_time'].dt.month
        data['day'] = data['date_time'].dt.day
        data['day_of_week'] = data['date_time'].dt.dayofweek
        data['day_of_year'] = data['date_time'].dt.dayofyear

        # Extract Time Features
        data['hour'] = data['date_time'].dt.hour
        data['minute'] = data['date_time'].dt.minute


        # Create a new DataFrame for the modified 'date_time' column
        new_data = data.copy()

        # Drop the 'date_time' column from the original DataFrame
        data.drop(columns=['date_time'], inplace=True)
        
        
        data = pd.get_dummies(data, columns=['weather_main'], prefix='', prefix_sep='')
        
        
        self.x = data.drop("traffic_volume",axis = 1)
        self.y = data["traffic_volume"]


    def train_test_spliting(self):
        # Assuming data_preprocessing has already been called
        if self.x is None or self.y is None:
            raise ValueError("Data must be preprocessed before splitting.")

        # Split the data into training and test sets. (0.75, 0.25) split.
        X_train, X_test, y_train, y_test = train_test_split(self.x, self.y, test_size=0.25, random_state=42)

        train = pd.concat([X_train, y_train], axis=1)
        test = pd.concat([X_test, y_test], axis=1)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Split data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

In [17]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.data_preprocessing()
    data_transformation.train_test_spliting()

except FileNotFoundError as file_not_found_error:
    logger.error(f"File not found: {file_not_found_error}")
except Exception as e:
    logger.error(f"An unexpected error occurred: {e}")
    raise e


[03/13/2024 19:03:36] 31 Trafficvol - INFO - yaml file: config\config.yaml loaded successfully
[03/13/2024 19:03:36] 31 Trafficvol - INFO - yaml file: params.yaml loaded successfully
[03/13/2024 19:03:36] 31 Trafficvol - INFO - yaml file: schema.yaml loaded successfully
[03/13/2024 19:03:36] 52 Trafficvol - INFO - created directory at: artifacts
[03/13/2024 19:03:36] 52 Trafficvol - INFO - created directory at: artifacts/data_transformation
[03/13/2024 19:03:36] 61 Trafficvol - INFO - Split data into training and test sets
[03/13/2024 19:03:36] 62 Trafficvol - INFO - (36135, 21)
[03/13/2024 19:03:36] 63 Trafficvol - INFO - (12045, 21)


(36135, 21)
(12045, 21)
